In [1]:
!pip install pandas matplotlib seaborn scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------------------- ------------------ 6.0/11.0 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 32.7 MB/s  0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------------------------------- - 7.9/8.1 MB 37.4 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 33.6 MB/s  0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -------------------------------- ------- 7.1/8.7 MB 39.6 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 33.9 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 31.8 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------------------ --------------- 7.9/12.8 MB 37.4 MB/s eta 0:00:01
   ---------------------------------------- 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# === STEP 1: Load your data ===
# Change file name and target column
df = pd.read_csv("your_data.csv")

target_col = "label"   # <- change to your label column name
X = df.drop(columns=[target_col])
y = df[target_col]

# === STEP 2: Mutual Information Ranking ===
mi = mutual_info_classif(X, y, random_state=42)
mi_rank = pd.Series(mi, index=X.columns).rank(ascending=False)

# === STEP 3: Recursive Feature Elimination (RFE) ===
estimator = RandomForestClassifier(random_state=42, n_estimators=100)
rfe = RFE(estimator, n_features_to_select=1, step=1)
rfe.fit(X, y)
rfe_rank = pd.Series(rfe.ranking_, index=X.columns).rank(ascending=True)

# === STEP 4: LightGBM Feature Importance ===
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X, y)
lgb_rank = pd.Series(lgb_model.feature_importances_, index=X.columns).rank(ascending=False)

# === STEP 5: Combine all rankings ===
rank_df = pd.DataFrame({
    "MI_Rank": mi_rank,
    "RFE_Rank": rfe_rank,
    "LGBM_Rank": lgb_rank
})
rank_df["Mean_Rank"] = rank_df.mean(axis=1)
rank_df = rank_df.sort_values("Mean_Rank")

# === STEP 6: Show Top Features ===
top_n = 10   # change this to how many you want
top_features = rank_df.head(top_n)

print("\nTop Ranked Features:")
print(top_features)

# === STEP 7: Save ranked features to CSV ===
rank_df.to_csv("feature_ranking.csv", index=True)
print("\nFull feature ranking saved to feature_ranking.csv")

# === STEP 8 (Optional): Reduce dataset ===
X_selected = X[top_features.index]
X_selected.to_csv("reduced_features.csv", index=False)
print(f"\nReduced dataset with top {top_n} features saved to reduced_features.csv")


KeyboardInterrupt: 